In [ ]:
import pickle
import datacardHelpers as helpers
from pathlib import Path
from HHbbVV.hh_vars import years, bg_keys
from HHbbVV.postprocessing.postprocessing import res_shape_vars, weight_shifts
import plotting

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with Path(
    "/ceph/cms/store/user/rkansal/bbVV/templates/25Feb8XHYFix/backgrounds/2016_templates.pkl"
).open("rb") as f:
    templates = pickle.load(f)

In [ ]:
templates_path = Path("/ceph/cms/store/user/rkansal/bbVV/templates/25Feb8XHYFix/")
sample = "NMSSM_XToYHTo2W2BTo4Q2B_MX-3000_MY-250"
templates_dict, templates_summed = helpers.get_templates(
    templates_path / "backgrounds", templates_path / sample, years, [sample], sig_separate=True
)

In [ ]:
h

In [ ]:
h.project(0, 1)["Z+Jets", ...]

In [ ]:
h.project(0, 1)["Z+Jets_L1EcalPrefiring_down", ...]

In [ ]:
sig_key = "X[3000]->H(bb)Y[250](VV)"
h = templates_dict["2016"]["pass"]
year = "2016"
ylims = [1, 0.2]
pbg_keys = [k for k in bg_keys if k != "QCD"]
wshift = "L1EcalPrefiring"
for i, shape_var in enumerate(res_shape_vars):
    plot_params = {
        "hists": h.project(0, i + 1),
        "sig_keys": [sig_key],
        "sig_scale_dict": {sig_key: 10},
        "show": True,
        "year": year,
        "ylim": ylims[i],
        "plot_data": False,
    }

    plotting.ratioHistPlot(
        **plot_params,
        bg_keys=pbg_keys,
        divide_bin_width=True,
        syst=(wshift, weight_shifts[wshift].samples),
        # title=title,
        # name=f"{plot_name}{jlabel}.pdf",
    )

    # if not do_jshift and plot_shifts:
    #     plot_name = (
    #         f"{plot_dir}/wshifts/" f"{split_str}{rname}_region_{shape_var.var}"
    #     )

    #     for wshift, wsyst in weight_shifts.items():
    #         plotting.ratioHistPlot(
    #             **plot_params,
    #             bg_keys=bg_keys,
    #             syst=(wshift, wsyst.samples),
    #             title=f"{region.label} Region {wsyst.label} Unc.",
    #             name=f"{plot_name}_{wshift}.pdf",
    #         )

    #         for skey, shift in [("Down", "down"), ("Up", "up")]:
    #             plotting.ratioHistPlot(
    #                 **plot_params,
    #                 bg_keys=p_bg_keys,  # don't plot QCD
    #                 syst=(wshift, wsyst.samples),
    #                 variation=shift,
    #                 title=f"{region.label} Region {wsyst.label} Unc. {skey} Shapes",
    #                 name=f"{plot_name}_{wshift}_{shift}.pdf",
    #                 plot_ratio=False,
    #             )

    #     if region.signal:
    #         plotting.ratioHistPlot(
    #             **plot_params,
    #             bg_keys=bg_keys,
    #             sig_err="txbb",
    #             title=rf"{region.label} Region $T_{{Xbb}}$ Shapes",
    #             name=f"{plot_name}_txbb.pdf",
    #         )

In [ ]:
print(f"Plotting templates: {time.time() - start:.2f}")
if plot_sig_keys is None:
    plot_sig_keys = sig_keys

if sig_scale_dict is None:
    sig_scale_dict = {
        **{skey: 1 for skey in nonres_sig_keys if skey in plot_sig_keys},
        **{skey: 1 for skey in res_sig_keys if skey in plot_sig_keys},
    }

title = (
    f"{region.label} Region Pre-Fit Shapes"
    if not do_jshift
    else f"{region.label} Region {jshift} Shapes"
)

if sig_splits is None:
    sig_splits = [plot_sig_keys]

# don't plot qcd in the pass regions
if pass_region:
    p_bg_keys = [key for key in bg_keys if key != qcd_key]
else:
    p_bg_keys = bg_keys

for i, shape_var in enumerate(shape_vars):
    for j, p_sig_keys in enumerate(sig_splits):
        split_str = "" if len(sig_splits) == 1 else f"sigs{j}_"
        plot_params = {
            "hists": h.project(0, i + 1),
            "sig_keys": p_sig_keys,
            "sig_scale_dict": (
                {key: sig_scale_dict[key] for key in p_sig_keys} if region.signal else None
            ),
            "show": show,
            "year": year,
            "ylim": pass_ylim if pass_region else fail_ylim,
            "plot_data": not (rname == "pass" and blind_pass),
        }

        plot_name = (
            f"{plot_dir}/"
            f"{'jshifts/' if do_jshift else ''}"
            f"{split_str}{rname}_region_{shape_var.var}"
        )

        plotting.ratioHistPlot(
            **plot_params,
            bg_keys=bg_keys,
            title=title,
            name=f"{plot_name}{jlabel}.pdf",
        )

        if not do_jshift and plot_shifts:
            plot_name = f"{plot_dir}/wshifts/" f"{split_str}{rname}_region_{shape_var.var}"

            for wshift, wsyst in weight_shifts.items():
                plotting.ratioHistPlot(
                    **plot_params,
                    bg_keys=bg_keys,
                    syst=(wshift, wsyst.samples),
                    title=f"{region.label} Region {wsyst.label} Unc.",
                    name=f"{plot_name}_{wshift}.pdf",
                )

                for skey, shift in [("Down", "down"), ("Up", "up")]:
                    plotting.ratioHistPlot(
                        **plot_params,
                        bg_keys=p_bg_keys,  # don't plot QCD
                        syst=(wshift, wsyst.samples),
                        variation=shift,
                        title=f"{region.label} Region {wsyst.label} Unc. {skey} Shapes",
                        name=f"{plot_name}_{wshift}_{shift}.pdf",
                        plot_ratio=False,
                    )

            if region.signal:
                plotting.ratioHistPlot(
                    **plot_params,
                    bg_keys=bg_keys,
                    sig_err="txbb",
                    title=rf"{region.label} Region $T_{{Xbb}}$ Shapes",
                    name=f"{plot_name}_txbb.pdf",
                )